## 3.2 케라스를 이용한 구현

### Sequential 모델

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

### Sequential 모델을 사용하는 경우

`Sequential` 모델은 각 레이어에 **정확히 하나의 입력 텐서와 하나의 출력 텐서**가 있는 **일반 레이어 스택**에 적합합니다.

개략적으로 다음과 같은 `Sequential` 모델은

In [2]:
model = keras.Sequential(
    [
        layers.Dense(2, activation="relu", name="layer1"),
        layers.Dense(3, activation="relu", name="layer2"),
        layers.Dense(4, name="layer3"),
    ]
)

x = tf.ones((3, 3))
y = model(x)

다음 함수와 동일합니다.

In [3]:
layer1 = layers.Dense(2, activation="relu", name="layer1")
layer2 = layers.Dense(3, activation="relu", name="layer2")
layer3 = layers.Dense(4, name="layer3")

x = tf.ones((3, 3))
y = layer3(layer2(layer1(x)))

Sequential 모델은 다음의 경우에 **적합하지 않습니다**.

- 모델에 다중 입력 또는 다중 출력이 있습니다
- 레이어에 다중 입력 또는 다중 출력이 있습니다
- 레이어 공유를 해야 합니다
- 비선형 토폴로지를 원합니다(예: 잔류 연결, 다중 분기 모델)

### Sequential 모델 생성하기

레이어의 목록을 Sequential 생성자에 전달하여 Sequential 모델을 만들 수 있습니다.

In [4]:
model = keras.Sequential(
    [
        layers.Dense(2, activation="relu"),
        layers.Dense(3, activation="relu"),
        layers.Dense(4),
    ]
)

속한 레이어는 `layers` 속성을 통해 접근할 수 있습니다.

In [5]:
model.layers

`add()` 메서드를 통해 Sequential 모델을 점진적으로 작성할 수도 있습니다.

In [6]:
model = keras.Sequential()
model.add(layers.Dense(2, activation="relu"))
model.add(layers.Dense(3, activation="relu"))
model.add(layers.Dense(4))

레이어를 제거하는 `pop()` 메서드도 있습니다. Sequential 모델은 레이어의 리스트와 매우 유사하게 동작합니다.

In [7]:
model.pop()
print(len(model.layers))  

2


또한 Sequential 생성자는 Keras의 모든 레이어 또는 모델과 마찬가지로 `name` 인수를 허용합니다. 이것은 의미론적으로 유의미한 이름으로 TensorBoard 그래프에 주석을 달 때 유용합니다.

In [8]:
model = keras.Sequential(name="my_sequential")
model.add(layers.Dense(2, activation="relu", name="layer1"))
model.add(layers.Dense(3, activation="relu", name="layer2"))
model.add(layers.Dense(4, name="layer3"))

### 미리 입력 형상 지정하기

일반적으로 Keras의 모든 레이어는 가중치를 만들려면 입력의 형상을 알아야 합니다. 따라서 다음과 같은 레이어를 만들면 처음에는 가중치가 없습니다.

In [9]:
layer = layers.Dense(3)
layer.weights  

[]

가중치는 모양이 입력의 형상에 따라 달라지기 때문에 입력에서 처음 호출될 때 가중치를 만듭니다.

In [10]:
x = tf.ones((1, 4))
y = layer(x)
layer.weights  

[<tf.Variable 'dense_6/kernel:0' shape=(4, 3) dtype=float32, numpy=
 array([[-0.4441235 ,  0.2121886 , -0.03958333],
        [ 0.4195751 , -0.09102201,  0.7704048 ],
        [-0.764909  , -0.27497667, -0.63498735],
        [-0.6983758 ,  0.5078819 , -0.17136121]], dtype=float32)>,
 <tf.Variable 'dense_6/bias:0' shape=(3,) dtype=float32, numpy=array([0., 0., 0.], dtype=float32)>]

당연히 이것은 Sequential 모델에도 적용됩니다. 입력 형상이 없는 Sequential 모델을 인스턴스화할 때는 "빌드"되지 않습니다. 가중치가 없습니다(그리고 `model.weights`를 호출하면 오류가 발생함). 모델에 처음 입력 데이터가 표시되면 가중치가 생성됩니다.

In [23]:
model = keras.Sequential(
    [
        layers.Dense(2, activation="relu"),
        layers.Dense(3, activation="relu"),
        layers.Dense(4),
    ]
)  

x = tf.ones((1, 4))
y = model(x)

for i, weight in enumerate(model.weights):
    print(f'weights[{i}] = {weight.name}\n{weight.numpy()}\n')

weights[0] = dense_43/kernel:0
[[-0.94046044 -0.00958014]
 [ 0.67068005  0.17483878]
 [ 0.50995994 -0.763247  ]
 [-0.05868912  0.33657622]]

weights[1] = dense_43/bias:0
[0. 0.]

weights[2] = dense_44/kernel:0
[[ 0.5630655  -0.62348694  0.9331384 ]
 [ 1.0583487   0.672593   -0.39086586]]

weights[3] = dense_44/bias:0
[0. 0. 0.]

weights[4] = dense_45/kernel:0
[[-0.8648149  -0.5868753  -0.9068501   0.1667881 ]
 [ 0.7519375  -0.5386523   0.5179932   0.12430656]
 [-0.5806255  -0.19936866  0.88470626  0.20879793]]

weights[5] = dense_45/bias:0
[0. 0. 0. 0.]



모델이 "빌드"되면, 그 내용을 표시하기 위해 `summary()` 메서드를 호출할 수 있습니다.

In [24]:
model.summary()

Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_43 (Dense)            (1, 2)                    10        
                                                                 
 dense_44 (Dense)            (1, 3)                    9         
                                                                 
 dense_45 (Dense)            (1, 4)                    16        
                                                                 
Total params: 35
Trainable params: 35
Non-trainable params: 0
_________________________________________________________________


그러나 현재 출력 형상을 포함하여 지금까지 모델의 요약을 표시할 수 있도록 Sequential 모델을 점진적으로 빌드할 때 매우 유용할 수 있습니다. 이 경우 `Input` 객체를 모델에 전달하여 모델의 시작 형상을 알 수 있도록 모델을 시작해야 합니다.

In [25]:
model = keras.Sequential()
model.add(keras.Input(shape=(4,)))
model.add(layers.Dense(2, activation="relu"))

model.summary()

Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_46 (Dense)            (None, 2)                 10        
                                                                 
Total params: 10
Trainable params: 10
Non-trainable params: 0
_________________________________________________________________


`Input` 객체는 레이어가 아니므로 `model.layers`의 일부로 표시되지 않습니다.

In [26]:
model.layers

간단한 대안은 첫 번째 레이어에 `input_shape` 인수를 전달하는 것입니다.

In [27]:
model = keras.Sequential()
model.add(layers.Dense(2, activation="relu", input_shape=(4,)))

model.summary()

Model: "sequential_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_47 (Dense)            (None, 2)                 10        
                                                                 
Total params: 10
Trainable params: 10
Non-trainable params: 0
_________________________________________________________________


이처럼 사전 정의된 입력 모양으로 빌드된 모델은 항상 가중치를 가지며(데이터를 보기 전에도) 항상 정의된 출력 형상을 갖습니다.

일반적으로 Sequential 모델의 입력 형상을 알고 있는 경우 항상 Sequential 모델의 입력 형상을 지정하는 것이 좋습니다.